In [245]:
# Библиотеки, которые пригодятся для решения

!pip install heapdict -q

import cvxpy as cvx
import itertools
import heapdict
import numpy as np
from numpy import unravel_index

# Задание на программирование [50 баллов]

Ваша задача --- реализовать алгоритм Branch and Bound для решения задачи судоку. Для удобства мы декомпозировали задачу на несколько функций, которые вам необхоимо реализовать. Пожалуйста, не меняйте имена функций. Если необходимо, вы можете создавать дополнительные вспомогательные функции.

**Пример CVXPY**

В этой задаче вам нужно будет использовать библиотеку CVXPY. Пример использования библиотеки можно найти в [Google Colab](https://colab.research.google.com/drive/13FjBaQvyMYQBbkoNLBW7Gdc07Oa3vAsZ)

**INPUT description**

Ваш алгоритм получит на вход один аргумент: судоку в виде двумерного списка 9 x 9, в котором нули обозначают пустые клетки. Например,

```
puzzle = [[4, 8, 0, 3, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 7, 1],
          [0, 2, 0, 0, 0, 0, 0, 0, 0],
          [7, 0, 5, 0, 0, 0, 0, 6, 0],
          [0, 0, 0, 2, 0, 0, 8, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 1, 0, 7, 6, 0, 0, 0],
          [3, 0, 0, 0, 0, 0, 4, 0, 0],
          [0, 0, 0, 0, 5, 0, 0, 0, 0]]
```

**OUTPUT description**

Ваш алгоритм должен вернуть две переменные.

`solved_puzzle`

Решение задачи судоку. Например,
```
solved_puzzle = [[4, 8, 7, 3, 1, 2, 6, 9, 5],
                 [5, 9, 3, 6, 8, 4, 2, 7, 1],
                 [1, 2, 6, 5, 9, 7, 3, 8, 4],
                 [7, 3, 5, 8, 4, 9, 1, 6, 2],
                 [9, 1, 4, 2, 6, 5, 8, 3, 7],
                 [2, 6, 8, 7, 3, 1, 5, 4, 9],
                 [8, 5, 1, 4, 7, 6, 9, 2, 3],
                 [3, 7, 9, 1, 2, 8, 4, 5, 6],
                 [6, 4, 2, 9, 5, 3, 7, 1, 8]]
```

`constraints`

Ограничения, добавление которых делает LP-релаксацию задачи судоку целочисленной. Напимер, ограничения вида

```
constraints = [(8, 5, 2, 1), (5, 3, 4, 1)]
```

говорят, что добавление дополнительных ограничений $(x_{8,5})_{2} = 1$ and $(x_{5,3})_{4} = 1$ позволяют решить задачу судоку с помощью симплекс алгоритма.

Подробнее постановка задачи описана в условии HW2.

## Основная функция

Решение судоку происходит с помощью функции `solve_sudoku`. Для ее реализации вам понадобятся вспомогательные функции, которые определены ниже (см. раздел **Вспомогательные функции**).

После того как вы реализуете алгоритм, вы можете протестировать его на нескольких тестах (см. раздел **Тестирование**).

In [246]:
def solve_sudoku(puzzle):
    """
    Основная функция решения судоку с помошью алгоритма Branch and Bound.

    Args:
    puzzle: Массив 9x9 с цифрами от 0 до 9. Цифра 0 означает пустую клетку

    Returns:
    Tuple (solved_puzzle, constraints) где:
    - solved_puzzle: Полностью заполненный массив 9x9 --- решение судоку
    - constraints: Список дополнительных ограничений, которые делают
                    LP-релаксацию задачи судоку целочисленной

    Hint: Внутри этой функции вам нужно будет использовать очередь с приоритетом
          для выбора направления исследования
    """

    # Your implementation here


    # У меня в итоге главной функцией стала solve_sudoku_auxiliary, и эта функция обращается к ней
    new_cond = []
    curr_puzzle = puzzle.copy()

    while True:
        curr_puzzle, curr_value, new_cond, flag = solve_sudoku_auxiliary(curr_puzzle, new_cond)
        #print(f'Я тут был и new_cond is {new_cond}')
        if flag:
            return curr_puzzle



## Вспомогательные функции

In [247]:
def generate_constraints(puzzle, variables):
    """
    Вам нужно задать ограничения для задачи судоку, используя cvxpy. Ограничения
    должны состоять из 4 компонентов:
    1. Каждая цифра (от 1 до 9) должна встречаться в каждой строке 1 раз
    2. Каждая цифра (от 1 до 9) должна встречаться в каждом столбце 1 раз
    3. Каждая цифра (от 1 до 9) должна встречаться в каждом из 9 квадратов 1 раз
    4. Цифры, которые даны в изначальной позиции, не должны быть изменены

    Args:
    puzzle: Массив 9x9 с цифрами от 0 до 9. Цифра 0 означает пустую клетку
    variables: Список 9 матриц бинарных переменных (cvxpy Variables). Каждая
               матрица относится к одной цифре (от 1 до 9). Если в матрице k
               значение (i, j) равно 1, то на позиции (i, j) в судоку стоит
               цифра k.

    Returns:
    constraints: Список ограничений (cvxpy constraints), которые задают правила
                 судоку.

    Hint 1: функция generate_constraints вызывается из функции
            solve_sudoku_auxiliary

    Hint 2: Чтобы задать группу ограничений 4, вы можете добавить ограничения
    вида x <= 1, x >= 1.
    """

    constraints = []

    # Your implementation here

    for i in range(len(variables)):
        for row in range(9): # Ограничения на строки
            constraints += [cvx.sum(variables[i][row]) == 1]
        for column in range(9): # Ограничения на столбцы
            constraints += [cvx.sum(variables[i][:, column]) == 1]

        for row in range(0, 9, 3): # Ограничения на квадраты
            for column in range(0, 9, 3):
                constraints += [cvx.sum(variables[i][row:row + 3, column:column + 3]) == 1]

    for row in range(9): # Ограничение, чтоб в каждой позиции было одно число
        for column in range(9):
            constraints += [cvx.sum(list((variables[i][row][column] for i in range(9)))) == 1]

    for i in range(len(variables)): # Ограничение на то, чтоб изначальные цифры не менялись
        for row in range(9):
            for column in range(9):
                if puzzle[row][column] == i + 1:
                    constraints += [variables[i][row][column] == 1]
                constraints += [variables[i][row][column] >= 0] # Ограничения, чтоб значения в матрицах были в [0, 1]
                constraints += [variables[i][row][column] <= 1]

    return constraints

In [248]:
def solve_sudoku_auxiliary(puzzle, const):
    """
    Вспомогательная функция для решения LP-релаксации задачи судоку. В случае,
    если решение LP-релаксации не дает целочисленного решения, функция определяет
    переменную, по которой можно сделать разбиение.

    Args:
    puzzle: Массив 9x9 с цифрами от 0 до 9. Цифра 0 означает пустую клетку
    const: Список дополнительных ограничений вида [row, column, value], которые
           должны быть дополнительно применены к судоку и возникают в процессе
           работы Branch and Bound.

    Returns:
    answer: Массив 9x9 с цифрами --- частичное или полное решение судоку. Если
            судоку полностью решено, то каждый элемент массива должен содержать
            число от 1 до 9
    value: значение целевой функции LP-релаксации
    index: Tuple (row, column, layer), который определяет, по какой переменной
           мы сделаем разбиение в следующей итерации Branch and Bound
    flag: Бинарный индикатор того, что судоку решено полностью (True) или
          алгоритму требуется дальнейший бренчинг (False)


    Hint: функция solve_sudoku_auxiliary вызывается из функции solve_sudoku.
    """

    variables = [cvx.Variable((9, 9)) for i in range(9)]
    objective = cvx.Minimize(cvx.sum(cvx.maximum(*[variables[i] for i in range(9)])))

    if len(const) == 3:
        puzzle[const[0], const[1]] = const[2]

    main_constraints = generate_constraints(puzzle, variables)


    # Your implementation here

    problem = cvx.Problem(objective, main_constraints)

    res = problem.solve()
    #print(res)

    new_puzzle = np.zeros((9, 9))
    for i in range(len(variables)): # Формируем судоку на текущем шаге
        temp = variables[i].value
        temp_mask = np.round(temp, 5) == 1
        new_puzzle += temp_mask * (i + 1)
        #print()
    #print(new_puzzle)

    cond_add = []

    hd = heapdict.heapdict()

    first_zero = np.argwhere(new_puzzle == 0) # Находим первый ноль в текущем судоку

    if len(first_zero) == 0: # Подгоночный иф, чтобы случайно не продолжать код с first_zero = None
        return new_puzzle, 0, 0, True
    else:
        first_zero = first_zero[0]

    for i in range(len(variables)):  # Ищем кандидатов на этот ноль
        #print(i + 1)
        #print(np.round(variables[i].value[first_zero[0], first_zero[1]], 5))

        if np.round(variables[i].value[first_zero[0], first_zero[1]], 5) == 0.5: # Если i + 1 кандидат на этот ноль (то есть там стоит 0.5)
             cond_add = np.array([first_zero[0], first_zero[1], i + 1]) # Берем новое условие

             # Дальше создаем все новое, чтобы cvx.variables от текущей задачи случайно не стали None
             new_variables = [cvx.Variable((9, 9)) for i in range(9)]
             new_objective = cvx.Minimize(cvx.sum(cvx.maximum(*[new_variables[i] for i in range(9)])))
             new_main_constraints = generate_constraints(puzzle, new_variables)
             new_problem = cvx.Problem(new_objective, new_main_constraints + [new_variables[i][first_zero[0]][first_zero[1]] == 1])
             new_res = new_problem.solve()
             #print(new_res)
             if new_res != np.inf:
                 hd[tuple(cond_add)] = new_res # Закидываем это условие в очередь


    return new_puzzle, res, hd.popitem()[0], len(new_puzzle[new_puzzle == 0]) == 0

In [249]:
arr = np.array([[4, 8, 0, 3, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 7, 1],
                [0, 2, 0, 0, 0, 0, 0, 0, 0],
                [7, 0, 5, 0, 0, 0, 0, 6, 0],
                [0, 0, 0, 2, 0, 0, 8, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 1, 0, 7, 6, 0, 0, 0],
                [3, 0, 0, 0, 0, 0, 4, 0, 0],
                [0, 0, 0, 0, 5, 0, 0, 0, 0]])
solve_sudoku(arr)

array([[4., 8., 7., 3., 1., 2., 6., 9., 5.],
       [5., 9., 3., 6., 8., 4., 2., 7., 1.],
       [1., 2., 6., 5., 9., 7., 3., 8., 4.],
       [7., 3., 5., 8., 4., 9., 1., 6., 2.],
       [9., 1., 4., 2., 6., 5., 8., 3., 7.],
       [2., 6., 8., 7., 3., 1., 5., 4., 9.],
       [8., 5., 1., 4., 7., 6., 9., 2., 3.],
       [3., 7., 9., 1., 2., 8., 4., 5., 6.],
       [6., 4., 2., 9., 5., 3., 7., 1., 8.]])

## Тестирование

Вы можете протестировать свой код на нескольких открытых тестах с помощью функции `test`.

Во время оценивания ваше решение будет дополнительно протестировано на закрытых тестах. Рекомендуем самостоятельно найти примеры судоку и проверить на них свое решение.


In [250]:
def test():
    """
    Функция для тестирования вашего решения.
    """

    sudoku_puzzle = [[8, 5, 9, 6, 1, 2, 4, 3, 7],
            [7, 2, 0, 8, 5, 4, 1, 6, 9],
            [1, 6, 4, 3, 7, 9, 5, 2, 8],
            [9, 8, 6, 1, 4, 7, 3, 5, 2],
            [3, 7, 5, 2, 6, 8, 9, 1, 4],
            [2, 4, 1, 5, 9, 0, 7, 8, 6],
            [4, 3, 2, 9, 8, 1, 6, 7, 5],
            [6, 1, 7, 0, 2, 5, 8, 9, 3],
            [5, 9, 8, 7, 3, 6, 2, 4, 1]]

    sudoku_sol = [[8, 5, 9, 6, 1, 2, 4, 3, 7],
            [7, 2, 3, 8, 5, 4, 1, 6, 9],
            [1, 6, 4, 3, 7, 9, 5, 2, 8],
            [9, 8, 6, 1, 4, 7, 3, 5, 2],
            [3, 7, 5, 2, 6, 8, 9, 1, 4],
            [2, 4, 1, 5, 9, 3, 7, 8, 6],
            [4, 3, 2, 9, 8, 1, 6, 7, 5],
            [6, 1, 7, 4, 2, 5, 8, 9, 3],
            [5, 9, 8, 7, 3, 6, 2, 4, 1]]

    print('=== solve_sudoku')
    print(' + Expected value: {}'.format((sudoku_sol, [])))
    print(' +     Your value: {}'.format(solve_sudoku(sudoku_puzzle)))

    sudoku_puzzle = [[8, 5, 0, 0, 0, 2, 4, 0, 0],
            [7, 2, 0, 0, 0, 0, 0, 0, 9],
            [0, 0, 4, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 1, 0, 7, 0, 0, 2],
            [3, 0, 5, 0, 0, 0, 9, 0, 0],
            [0, 4, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 8, 0, 0, 7, 0],
            [0, 1, 7, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 3, 6, 0, 4, 0]]

    sudoku_sol = [[8, 5, 9, 6, 1, 2, 4, 3, 7],
            [7, 2, 3, 8, 5, 4, 1, 6, 9],
            [1, 6, 4, 3, 7, 9, 5, 2, 8],
            [9, 8, 6, 1, 4, 7, 3, 5, 2],
            [3, 7, 5, 2, 6, 8, 9, 1, 4],
            [2, 4, 1, 5, 9, 3, 7, 8, 6],
            [4, 3, 2, 9, 8, 1, 6, 7, 5],
            [6, 1, 7, 4, 2, 5, 8, 9, 3],
            [5, 9, 8, 7, 3, 6, 2, 4, 1]]

    print('=== solve_sudoku')
    print(' + Expected value: {}'.format((sudoku_sol, [])))
    print(' +     Your value: {}'.format(solve_sudoku(sudoku_puzzle)))

    sudoku_puzzle = [[4, 8, 0, 3, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 7, 1],
            [0, 2, 0, 0, 0, 0, 0, 0, 0],
            [7, 0, 5, 0, 0, 0, 0, 6, 0],
            [0, 0, 0, 2, 0, 0, 8, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 1, 0, 7, 6, 0, 0, 0],
            [3, 0, 0, 0, 0, 0, 4, 0, 0],
            [0, 0, 0, 0, 5, 0, 0, 0, 0]]

    sudoku_sol = [[4, 8, 7, 3, 1, 2, 6, 9, 5],
            [5, 9, 3, 6, 8, 4, 2, 7, 1],
            [1, 2, 6, 5, 9, 7, 3, 8, 4],
            [7, 3, 5, 8, 4, 9, 1, 6, 2],
            [9, 1, 4, 2, 6, 5, 8, 3, 7],
            [2, 6, 8, 7, 3, 1, 5, 4, 9],
            [8, 5, 1, 4, 7, 6, 9, 2, 3],
            [3, 7, 9, 1, 2, 8, 4, 5, 6],
            [6, 4, 2, 9, 5, 3, 7, 1, 8]]

    print('=== solve_sudoku')
    print(' + Expected value: {}'.format((sudoku_sol, [(6, 3, 2, 0)])))
    print(' +     Your value: {}'.format(solve_sudoku(sudoku_puzzle)))

test()

=== solve_sudoku
 + Expected value: ([[8, 5, 9, 6, 1, 2, 4, 3, 7], [7, 2, 3, 8, 5, 4, 1, 6, 9], [1, 6, 4, 3, 7, 9, 5, 2, 8], [9, 8, 6, 1, 4, 7, 3, 5, 2], [3, 7, 5, 2, 6, 8, 9, 1, 4], [2, 4, 1, 5, 9, 3, 7, 8, 6], [4, 3, 2, 9, 8, 1, 6, 7, 5], [6, 1, 7, 4, 2, 5, 8, 9, 3], [5, 9, 8, 7, 3, 6, 2, 4, 1]], [])
 +     Your value: [[8. 5. 9. 6. 1. 2. 4. 3. 7.]
 [7. 2. 3. 8. 5. 4. 1. 6. 9.]
 [1. 6. 4. 3. 7. 9. 5. 2. 8.]
 [9. 8. 6. 1. 4. 7. 3. 5. 2.]
 [3. 7. 5. 2. 6. 8. 9. 1. 4.]
 [2. 4. 1. 5. 9. 3. 7. 8. 6.]
 [4. 3. 2. 9. 8. 1. 6. 7. 5.]
 [6. 1. 7. 4. 2. 5. 8. 9. 3.]
 [5. 9. 8. 7. 3. 6. 2. 4. 1.]]
=== solve_sudoku
 + Expected value: ([[8, 5, 9, 6, 1, 2, 4, 3, 7], [7, 2, 3, 8, 5, 4, 1, 6, 9], [1, 6, 4, 3, 7, 9, 5, 2, 8], [9, 8, 6, 1, 4, 7, 3, 5, 2], [3, 7, 5, 2, 6, 8, 9, 1, 4], [2, 4, 1, 5, 9, 3, 7, 8, 6], [4, 3, 2, 9, 8, 1, 6, 7, 5], [6, 1, 7, 4, 2, 5, 8, 9, 3], [5, 9, 8, 7, 3, 6, 2, 4, 1]], [])
 +     Your value: [[8. 5. 9. 6. 1. 2. 4. 3. 7.]
 [7. 2. 3. 8. 5. 4. 1. 6. 9.]
 [1. 6. 4. 3. 7. 9. 5. 